# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'12-22-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'12-22-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-12-23 05:22:03,33.93911,67.709953,50190,2096,39585,8509.0,Afghanistan,128.929204,4.176131
1,NaN,NaN,NaN,Albania,2020-12-23 05:22:03,41.15330,20.168300,53814,1111,29249,23454.0,Albania,1869.970116,2.064519
2,NaN,NaN,NaN,Algeria,2020-12-23 05:22:03,28.03390,1.659600,96069,2687,64020,29362.0,Algeria,219.080308,2.796948
3,NaN,NaN,NaN,Andorra,2020-12-23 05:22:03,42.50630,1.521800,7633,82,7073,478.0,Andorra,9878.987899,1.074283
4,NaN,NaN,NaN,Angola,2020-12-23 05:22:03,-11.20270,17.873900,16802,393,9729,6680.0,Angola,51.122324,2.339007


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,48527,48718,48952,49161,49378,49621,49681,49817,50013,50190
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,48530,49191,50000,50637,51424,52004,52542,53003,53425,53814
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,92102,92597,93065,93507,93933,94371,94781,95203,95659,96069


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1965,1969,1995,2011,2025,2030,2047,2067,2082,2096
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,1003,1016,1028,1040,1055,1066,1074,1088,1098,1111
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,2596,2609,2623,2631,2640,2647,2659,2666,2675,2687


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,38250,38252,38336,38475,38505,38540,38613,39006,39508,39585
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,24820,24820,25876,26381,26898,27426,27831,28121,28752,29249
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,60457,60888,61307,61700,62089,62487,62869,63260,63644,64020


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,3300,3329,3426,3510,3570,3647,3698,3741,3780,3841
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,10806,10898,11061,11212,11364,11556,11722,11827,11952,12155
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1272,1275,1292,1296,1309,1318,1330,1336,1336,1363


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,41,41,43,43,43,44,44,44,44,44
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,141,141,141,145,145,146,147,147,147,149
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,30,30,30,30,30,30,32,32,32,32


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incident_Rate,Case_Fatality_Ratio
649,1001.0,Autauga,Alabama,US,2020-12-23 05:22:03,32.539527,-86.644082,3841,44,0,3797.0,"Autauga, Alabama, US",6875.011187,1.145535
686,1075.0,Lamar,Alabama,US,2020-12-23 05:22:03,33.779950,-88.096680,979,12,0,967.0,"Lamar, Alabama, US",7091.633466,1.225741
687,1077.0,Lauderdale,Alabama,US,2020-12-23 05:22:03,34.901719,-87.656247,5722,65,0,5657.0,"Lauderdale, Alabama, US",6170.669370,1.135966


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,18230242,322765,0
India,10099066,146444,9663382
Brazil,7318821,188259,6501341


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,18230242,322765,0,17907477,2020-12-23 05:22:03,37.936303,-91.379001
India,10099066,146444,9663382,289240,2020-12-23 05:22:03,23.088275,81.806127
Brazil,7318821,188259,6501341,629221,2020-12-23 05:22:03,-12.669522,-48.480493
Russia,2878382,51274,2297289,529819,2020-12-23 05:22:03,54.546312,62.120860
France,2547577,61821,191892,2293864,2020-12-23 05:22:03,8.314863,-21.695174


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,1958508,23239,0
Texas,1626614,26117,0
Florida,1223015,20754,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,1958508,23239,0,1935269,2020-12-23 05:22:03,37.843962,-120.728594
Texas,1626614,26117,0,1600497,2020-12-23 05:22:03,31.660643,-98.653069
Florida,1223015,20754,0,1202261,2020-12-23 05:22:03,28.940755,-82.700744
Illinois,911308,16671,0,894637,2020-12-23 05:22:03,39.843618,-89.178330
New York,866765,36724,0,830041,2020-12-23 05:22:03,42.544151,-75.474183


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,647542,9016,0
Illinois,Cook,375360,7853,0
Arizona,Maricopa,287646,4754,0
Florida,Miami-Dade,278739,4095,0
Texas,Harris,222931,3260,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,647542,9016,0,638526,2020-12-23 05:22:03,34.308284,-118.228241,6037.0
Illinois,Cook,375360,7853,0,367507,2020-12-23 05:22:03,41.841448,-87.816588,17031.0
Arizona,Maricopa,287646,4754,0,282892,2020-12-23 05:22:03,33.348359,-112.491815,4013.0
Florida,Miami-Dade,278739,4095,0,274644,2020-12-23 05:22:03,25.611236,-80.551706,12086.0
Texas,Harris,222931,3260,0,219671,2020-12-23 05:22:03,29.858649,-95.393395,48201.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-20,49817,53003,95203,7577,16644,153,1541285,153825,28198,338854,...,2046161,13048,75933,1,110075,1413,122643,2087,18716,12325
2020-12-21,50013,53425,95659,7602,16686,153,1547138,154065,28219,340373,...,2079678,13477,76048,1,110513,1414,123945,2087,18768,12422
2020-12-22,50190,53814,96069,7633,16802,153,1555279,154602,28237,342226,...,2116609,14001,76180,1,110828,1420,125506,2087,18881,12544


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-20,2067,1088,2666,81,387,5,41813,2630,908,5351,...,67503,119,612,0,988,35,1141,606,373,320
2020-12-21,2082,1098,2675,81,390,5,41997,2656,908,5435,...,67718,119,612,0,993,35,1169,606,375,322
2020-12-22,2096,1111,2687,82,393,5,42254,2673,908,5540,...,68409,120,612,0,997,35,1198,606,379,326


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-20,39006,28121,63260,6997,9592,143,1368346,132532,25709,303213,...,4375,8327,73242,1,104462,1269,97979,1384,17762,10024
2020-12-21,39508,28752,63644,7028,9695,143,1374401,133176,25716,305658,...,4386,8688,73339,1,104799,1269,99547,1384,17818,10112
2020-12-22,39585,29249,64020,7073,9729,144,1379726,134586,25718,309595,...,4409,9127,73460,1,104799,1281,101355,1384,17890,10159


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,3300,3329,3426,3510,3570,3647,3698,3741,3780,3841
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,855,858,870,889,912,925,941,946,966,979
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,5078,5125,5200,5243,5346,5428,5531,5582,5644,5722
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,1585,1596,1617,1635,1672,1708,1739,1757,1783,1819
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,8744,8788,8857,8966,9055,9188,9283,9337,9372,9490


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-20,322452,43871,0,453597,201650,1884033,308890,162782,50616,49,...,519687,1593471,250964,6443,1921,306848,222600,72342,492724,41662
2020-12-21,324832,44023,0,461345,203107,1923887,311036,167377,51056,49,...,529578,1604991,252783,6534,1921,310890,226635,73337,494353,42118
2020-12-22,329811,44463,0,467215,205048,1958508,313552,168960,51623,49,...,534019,1626614,255085,6608,1934,314481,227887,74737,497380,42379


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                      \
Admin2         Autauga Baldwin Barbour  Bibb Blount Bullock Butler Calhoun   
2020-12-20        3741   11827    1336  1613   4218     753   1399    8648   
2020-12-21        3780   11952    1336  1628   4234     754   1405    8684   
2020-12-22        3841   12155    1363  1660   4313     760   1412    8856   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-12-20         2125     1287  ...    1784    410     2401      565   
2020-12-21         2133     1294  ...    1812    412     2420      567   
2020-12-22         2161     1314  ...    1842    415     2426      575   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-12-20           2718  2002  1416          0      699    446  
2020-12-21           2762  2019  1446          0      713    447  
2020-12-22           2780  2033  1460          0      720    455  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-20,4389,184,0,7971,3237,22675,4368,5581,862,0,...,6071,25827,1155,111,23,4650,3104,1128,4743,351
2020-12-21,4389,184,0,7972,3295,22837,4378,5676,871,0,...,6136,25936,1161,111,23,4654,3106,1129,4751,351
2020-12-22,4452,194,0,8125,3338,23239,4369,5703,872,0,...,6269,26117,1173,112,23,4705,3131,1171,4879,373


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-12-20          44     147      32   42     54      20     45     140   
2020-12-21          44     147      32   42     54      20     45     140   
2020-12-22          44     149      32   42     54      20     45     141   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-12-20           60       23  ...       7      8       15        5   
2020-12-21           60       23  ...       7      8       15        5   
2020-12-22           61       23  ...       7      8       15        5   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-12-20             15     2     7          0       11      2  
2020-12-21             15     2     7          0       11      2  
2020-12-22             15     2     7          0       12      2  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-20,0.002737,0.008774,0.004452,0.002249,0.001083,0.006579,0.002678,0.004257,0.001065,0.004878,...,0.017952,0.039102,0.001675,0.0,0.002678,0.001417,0.011772,0.0,0.005156,0.014320
2020-12-21,0.003934,0.007962,0.004790,0.003299,0.002523,0.000000,0.003797,0.001560,0.000745,0.004483,...,0.016380,0.032879,0.001514,0.0,0.003979,0.000708,0.010616,0.0,0.002778,0.007870
2020-12-22,0.003539,0.007281,0.004286,0.004078,0.006952,0.000000,0.005262,0.003486,0.000638,0.005444,...,0.017758,0.038881,0.001736,0.0,0.002850,0.004243,0.012594,0.0,0.006021,0.009821


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-20,0.009770,0.013035,0.002633,0.012500,0.002591,0.0,0.001197,0.005352,0.0,0.027261,...,0.004853,0.043860,0.0,NaN,0.004065,0.0,0.021486,0.0,0.000000,0.006289
2020-12-21,0.007257,0.009191,0.003376,0.000000,0.007752,0.0,0.004401,0.009886,0.0,0.015698,...,0.003185,0.000000,0.0,NaN,0.005061,0.0,0.024540,0.0,0.005362,0.006250
2020-12-22,0.006724,0.011840,0.004486,0.012346,0.007692,0.0,0.006119,0.006401,0.0,0.019319,...,0.010204,0.008403,0.0,NaN,0.004028,0.0,0.024808,0.0,0.010667,0.012422


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-20,0.010178,0.010420,0.006219,0.004883,0.007775,0.014184,0.004204,0.004555,0.000195,0.008656,...,0.004823,0.040615,0.001956,0.0,0.002437,0.000000,0.015737,0.0,0.000338,0.004006
2020-12-21,0.012870,0.022439,0.006070,0.004430,0.010738,0.000000,0.004425,0.004859,0.000272,0.008064,...,0.002514,0.043353,0.001324,0.0,0.003226,0.000000,0.016003,0.0,0.003153,0.008779
2020-12-22,0.001949,0.017286,0.005908,0.006403,0.003507,0.006993,0.003874,0.010587,0.000078,0.012880,...,0.005244,0.050529,0.001650,0.0,0.000000,0.009456,0.018162,0.0,0.004041,0.004648


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-20,0.007965,0.004189,NaN,0.011972,0.007676,0.022510,0.007476,0.000000,0.009574,0.0,...,0.031840,0.005848,0.008009,0.015765,0.005759,0.012793,0.000000,0.015825,0.004168,0.004194
2020-12-21,0.007381,0.003465,NaN,0.017081,0.007225,0.021154,0.006947,0.028228,0.008693,0.0,...,0.019033,0.007230,0.007248,0.014124,0.000000,0.013173,0.018127,0.013754,0.003306,0.010945
2020-12-22,0.015328,0.009995,NaN,0.012724,0.009557,0.017995,0.008089,0.009458,0.011105,0.0,...,0.008386,0.013472,0.009107,0.011325,0.006767,0.011551,0.005524,0.019090,0.006123,0.006197


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-12-20      0.011628  0.008958  0.004511  0.007495  0.006442  0.001330   
2020-12-21      0.010425  0.010569  0.000000  0.009299  0.003793  0.001328   
2020-12-22      0.016138  0.016985  0.020210  0.019656  0.018658  0.007958   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-12-20      0.004307  0.006283  0.004253  0.007042  ...  0.007340   
2020-12-21      0.004289  0.004163  0.003765  0.005439  ...  0.015695   
2020-12-22      0.004982  0.019807  0.013127  0.015456  ...  0.016556   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-12-20      0.007371  0.002924  0.003552   0.005922  0.005020  0.007112   
2020-12-21      0.004878  0.007913  0.003540   0.016188  0.008492  0.021186   
2020-12-22      0.007282  0.002479  0.014109   0.006517  0.006934  0.009682   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-12-20            NaN  0.005755  0.006772  
2020-12-21            NaN  0.020029  0.002242  
2020-12-22            NaN  0.009818  0.017897  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-20,0.000000,0.000000,NaN,0.004284,0.014416,0.004118,0.006684,0.000000,0.004662,NaN,...,0.018624,0.004394,0.006098,0.027778,0.0,0.001508,0.000000,0.005348,0.004447,0.000000
2020-12-21,0.000000,0.000000,NaN,0.000125,0.017918,0.007144,0.002289,0.017022,0.010441,NaN,...,0.010707,0.004220,0.005195,0.000000,0.0,0.000860,0.000644,0.000887,0.001687,0.000000
2020-12-22,0.014354,0.054348,NaN,0.019192,0.013050,0.017603,-0.002056,0.004757,0.001148,NaN,...,0.021675,0.006979,0.010336,0.009009,0.0,0.010958,0.008049,0.037201,0.026942,0.062678


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State Alabama                                                         \
Admin2         Autauga   Baldwin Barbour Bibb Blount Bullock Butler   Calhoun   
2020-12-20         0.0  0.000000     0.0  0.0    0.0     0.0    0.0  0.000000   
2020-12-21         0.0  0.000000     0.0  0.0    0.0     0.0    0.0  0.000000   
2020-12-22         0.0  0.013605     0.0  0.0    0.0     0.0    0.0  0.007143   

Province_State                     ... Wyoming                           \
Admin2          Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-12-20      0.000000      0.0  ...     0.0    0.0      0.0      0.0   
2020-12-21      0.000000      0.0  ...     0.0    0.0      0.0      0.0   
2020-12-22      0.016667      0.0  ...     0.0    0.0      0.0      0.0   

Province_State                                                     
Admin2         Sweetwater Teton Uinta Unassigned  Washakie Weston  
2020-12-20            0.0   0.0   0.0        NaN  0.000000    0.0  
2020-12-21            0.0   0.0   0.0        NaN  0.000000    0.0  
2020-12-22            0.0   0.0   0.0        NaN  0.090909    0.0  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-20,0.002835,0.010252,0.004492,0.003912,0.002636,0.004772,0.003426,0.005188,0.001125,0.005919,...,0.016138,0.043654,0.001679,0.0,0.002973,0.001322,0.013519,0.000060,0.004009,0.012800
2020-12-21,0.003385,0.009107,0.004641,0.003606,0.002580,0.002386,0.003612,0.003374,0.000935,0.005201,...,0.016259,0.038266,0.001597,0.0,0.003476,0.001015,0.012067,0.000030,0.003394,0.010335
2020-12-22,0.003462,0.008194,0.004464,0.003842,0.004766,0.001193,0.004437,0.003430,0.000786,0.005322,...,0.017009,0.038574,0.001666,0.0,0.003163,0.002629,0.012331,0.000015,0.004707,0.010078


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-20,0.008237,0.011356,0.003247,0.007883,0.004430,0.000977,0.002073,0.006562,2.628650e-10,0.025573,...,0.006358,0.042112,1.601737e-06,NaN,0.004479,6.127582e-35,0.020769,-0.000007,0.001501,0.006144
2020-12-21,0.007747,0.010274,0.003311,0.003941,0.006091,0.000488,0.003237,0.008224,1.314325e-10,0.020636,...,0.004771,0.021056,8.008686e-07,NaN,0.004770,3.063791e-35,0.022655,-0.000003,0.003432,0.006197
2020-12-22,0.007236,0.011057,0.003899,0.008143,0.006892,0.000244,0.004678,0.007312,6.571626e-11,0.019977,...,0.007488,0.014730,4.004343e-07,NaN,0.004399,1.531896e-35,0.023731,-0.000002,0.007049,0.009310


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
2020-12-20,0.005884,0.013978,0.006257,0.005995,0.010977,0.007779,0.003979,0.006050,0.000161,0.009775,...,0.012786,0.039789,0.001896,0.0,0.002388,0.001566,0.017829,-0.000007,0.001414,0.006435
2020-12-21,0.009377,0.018208,0.006164,0.005213,0.010858,0.003890,0.004202,0.005455,0.000217,0.008919,...,0.007650,0.041571,0.001610,0.0,0.002807,0.000783,0.016916,-0.000004,0.002284,0.007607
2020-12-22,0.005663,0.017747,0.006036,0.005808,0.007182,0.005441,0.004038,0.008021,0.000147,0.010900,...,0.006447,0.046050,0.001630,0.0,0.001404,0.005120,0.017539,-0.000002,0.003162,0.006127


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-20,0.011208,0.006810,NaN,0.013031,0.010792,0.023793,0.008752,0.004057,0.013040,3.504720e-85,...,0.021086,0.008223,0.009299,0.016188,0.007669,0.012396,0.005554,0.018159,0.006300,0.006139
2020-12-21,0.009295,0.005137,NaN,0.015056,0.009009,0.022473,0.007850,0.016142,0.010866,1.752360e-85,...,0.020059,0.007726,0.008274,0.015156,0.003835,0.012785,0.011841,0.015957,0.004803,0.008542
2020-12-22,0.012311,0.007566,NaN,0.013890,0.009283,0.020234,0.007969,0.012800,0.010986,8.761800e-86,...,0.014223,0.010599,0.008690,0.013241,0.005301,0.012168,0.008682,0.017523,0.005463,0.007369


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-12-20      0.014461  0.011875  0.006371  0.012118  0.012603  0.003907   
2020-12-21      0.012443  0.011222  0.003186  0.010709  0.008198  0.002618   
2020-12-22      0.014290  0.014103  0.011698  0.015182  0.013428  0.005288   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-12-20      0.006740  0.012114  0.007042  0.010072  ...  0.009627   
2020-12-21      0.005514  0.008138  0.005403  0.007756  ...  0.012661   
2020-12-22      0.005248  0.013972  0.009265  0.011606  ...  0.014609   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-12-20      0.007417  0.003343  0.004635   0.008473  0.004834  0.008437   
2020-12-21      0.006148  0.005628  0.004087   0.012331  0.006663  0.014812   
2020-12-22      0.006715  0.004054  0.009098   0.009424  0.006799  0.012247   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-12-20           -1.0  0.010389  0.005376  
2020-12-21           -1.0  0.015209  0.003809  
2020-12-22           -1.0  0.012513  0.010853  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-12-20,0.008155,0.002939,NaN,0.010100,0.014164,0.007654,0.011247,0.001599,0.006219,NaN,...,0.014441,0.006974,0.009009,0.021016,4.921512e-16,0.005565,0.002100,0.015604,0.010870,0.004809
2020-12-21,0.004078,0.001469,NaN,0.005113,0.016041,0.007399,0.006768,0.009310,0.008330,NaN,...,0.012574,0.005597,0.007102,0.010508,2.460756e-16,0.003213,0.001372,0.008245,0.006278,0.002405
2020-12-22,0.009216,0.027909,NaN,0.012152,0.014546,0.012501,0.002356,0.007034,0.004739,NaN,...,0.017125,0.006288,0.008719,0.009759,1.230378e-16,0.007086,0.004711,0.022723,0.016610,0.032541


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                          \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   
2020-12-20      0.003623  0.003464  0.016674  0.002030  0.013424   
2020-12-21      0.001811  0.001732  0.008337  0.001015  0.006712   
2020-12-22      0.000906  0.007669  0.004169  0.000507  0.003356   

Province_State                                                        ...  \
Admin2               Bullock    Butler   Calhoun  Chambers  Cherokee  ...   
2020-12-20      4.015472e-07  0.002887  0.008188  0.009793 -0.005208  ...   
2020-12-21      2.007736e-07  0.001444  0.004094  0.004897 -0.002604  ...   
2020-12-22      1.003868e-07  0.000722  0.005618  0.010782 -0.001302  ...   

Province_State   Wyoming                                              \
Admin2              Park        Platte  Sheridan Sublette Sweetwater   
2020-12-20      0.000010  1.394696e-07  0.009642  0.00036   0.001244   
2020-12-21      0.000005  6.973480e-08  0.004821  0.00018   0.000622   
2020-12-22      0.000003  3.486740e-08  0.002410  0.00009   0.000311   

Province_State                                                             
Admin2                 Teton     Uinta Unassigned  Washakie        Weston  
2020-12-20      3.552714e-15  0.000054  -0.501961  0.000040  9.536742e-07  
2020-12-21      1.776357e-15  0.000027  -0.501961  0.000020  4.768371e-07  
2020-12-22      8.881784e-16  0.000013  -0.501961  0.045464  2.384186e-07  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201222,AK,43001,NaN,1172281,NaN,totalTestsViral,1215282,121.0,948.0,...,1215282,10,11,ed10d732e9e31c3e61fc1b69fc99ab1008ea536b,0,0,0,0,0,NaN
1,20201222,AL,329811,61472.0,1532640,NaN,totalTestsPeopleViral,1800979,2527.0,31305.0,...,1862451,63,311,927ec7e6415554d071b9d32b5645db82830d4828,0,0,0,0,0,NaN
2,20201222,AR,205048,32266.0,1785196,NaN,totalTestsViral,1957978,1103.0,10651.0,...,1990244,43,81,18dc093f155ec5f991d2f62bee72ff6861c13d5e,0,0,0,0,0,NaN
3,20201222,AS,0,NaN,2140,NaN,totalTestsViral,2140,NaN,NaN,...,2140,0,0,ae2cf01a38710e4c59dba20bb87c5f658befdf1c,0,0,0,0,0,NaN
4,20201222,AZ,467215,20118.0,2233751,NaN,totalTestsViral,4690016,4019.0,33619.0,...,2700966,153,424,1c76275f04b9d0f76262c9104a9498efbcae1d39,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

/Users/Brian/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201222,AK,43001.0,NaN,1172281.0,NaN,totalTestsViral,1215282.0,121.0,948.0,...,1215282,10,11,ed10d732e9e31c3e61fc1b69fc99ab1008ea536b,0,0,0,0,0,NaN
1,20201222,AL,329811.0,61472.0,1532640.0,NaN,totalTestsPeopleViral,1800979.0,2527.0,31305.0,...,1862451,63,311,927ec7e6415554d071b9d32b5645db82830d4828,0,0,0,0,0,NaN
2,20201222,AR,205048.0,32266.0,1785196.0,NaN,totalTestsViral,1957978.0,1103.0,10651.0,...,1990244,43,81,18dc093f155ec5f991d2f62bee72ff6861c13d5e,0,0,0,0,0,NaN
3,20201222,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,...,2140,0,0,ae2cf01a38710e4c59dba20bb87c5f658befdf1c,0,0,0,0,0,NaN
4,20201222,AZ,467215.0,20118.0,2233751.0,NaN,totalTestsViral,4690016.0,4019.0,33619.0,...,2700966,153,424,1c76275f04b9d0f76262c9104a9498efbcae1d39,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-12-22,AK,43001,NaN,1172281,NaN,totalTestsViral,1215282,121.0,948.0,NaN,...,1215282,10,11,ed10d732e9e31c3e61fc1b69fc99ab1008ea536b,0,0,0,0,0,NaN
2020-12-22,AL,329811,61472.0,1532640,NaN,totalTestsPeopleViral,1800979,2527.0,31305.0,NaN,...,1862451,63,311,927ec7e6415554d071b9d32b5645db82830d4828,0,0,0,0,0,NaN
2020-12-22,AR,205048,32266.0,1785196,NaN,totalTestsViral,1957978,1103.0,10651.0,353.0,...,1990244,43,81,18dc093f155ec5f991d2f62bee72ff6861c13d5e,0,0,0,0,0,NaN
2020-12-22,AS,0,NaN,2140,NaN,totalTestsViral,2140,NaN,NaN,NaN,...,2140,0,0,ae2cf01a38710e4c59dba20bb87c5f658befdf1c,0,0,0,0,0,NaN
2020-12-22,AZ,467215,20118.0,2233751,NaN,totalTestsViral,4690016,4019.0,33619.0,943.0,...,2700966,153,424,1c76275f04b9d0f76262c9104a9498efbcae1d39,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-12-22,AK,43001.0,NaN,1172281.0,NaN,totalTestsViral,1215282.0,121.0,948.0,NaN,...,1215282,10,11,ed10d732e9e31c3e61fc1b69fc99ab1008ea536b,0,0,0,0,0,NaN
2020-12-22,AL,329811.0,61472.0,1532640.0,NaN,totalTestsPeopleViral,1800979.0,2527.0,31305.0,NaN,...,1862451,63,311,927ec7e6415554d071b9d32b5645db82830d4828,0,0,0,0,0,NaN
2020-12-22,AR,205048.0,32266.0,1785196.0,NaN,totalTestsViral,1957978.0,1103.0,10651.0,353.0,...,1990244,43,81,18dc093f155ec5f991d2f62bee72ff6861c13d5e,0,0,0,0,0,NaN
2020-12-22,AS,0.0,NaN,2140.0,NaN,totalTestsViral,2140.0,NaN,NaN,NaN,...,2140,0,0,ae2cf01a38710e4c59dba20bb87c5f658befdf1c,0,0,0,0,0,NaN
2020-12-22,AZ,467215.0,20118.0,2233751.0,NaN,totalTestsViral,4690016.0,4019.0,33619.0,943.0,...,2700966,153,424,1c76275f04b9d0f76262c9104a9498efbcae1d39,0,0,0,0,0,NaN
